In [1]:
import streamlit as st
import numpy as np
import pandas as pd
import numpy as np
import requests
import json
import os
import datetime
import dotenv 

%config InlineBackend.figure_format = 'retina'
%matplotlib inline

In [ ]:
    #     dotenv.load_dotenv()
    #     api_key_google = os.getenv('api_key') 
    #     gmaps = googlemaps.Client(key=api_key_google)

    #     params = {
    #     'origin': selected_starting_point,
    #     'destination': selected_destination,
    #     'mode': 'driving',  # You can change this to 'walking', 'bicycling', or 'transit'
    #     'alternatives': True,  # If you want alternative routes
    #     'avoid': None,  # You can specify features to avoid, e.g., 'tolls', 'highways'
    #     'language': 'en',  # Language for the results
    #     'units': 'imperial',  # Use 'metric' for metric units
    #     'departure_time': selected_departure,  
    # }

        # try:
        #     directions = gmaps.directions(**params)
        # except googlemaps.exceptions.ApiError as e:
        #     st.error(f"Google Maps API Error: {e}")
        #     return

        # route_df = pd.json_normalize(directions[0], record_path=['legs', 'steps'])

        # route_df['time_in_minutes'] = round(route_df['duration.value']/60, 0)
        # route_df['time_in_minutes'] = route_df['time_in_minutes'].astype('int')
        # route_df['cumulative_time'] = route_df['time_in_minutes'].cumsum()

        # route_df['date_time'] = route_df['cumulative_time'].apply(lambda x: selected_departure + timedelta(minutes=x))
        # route_df['date_time'] = route_df['date_time'].apply(lambda x: x.replace(minute=0, second=0, microsecond=0))

        # max_time = route_df['cumulative_time'].iloc[(len(route_df)-1)]
        # desired_val = list(range(0,max_time+60, 60))

        # route_df[['end_location.lat', 'end_location.lng', 'cumulative_time', 'date_time']]

        # closest_rows = (route_df.loc[[abs(route_df['cumulative_time'] - hour).idxmin() for hour in desired_val]]).reset_index()
        # #st.text(closest_rows)

        #return closest_rows

In [8]:
r = requests.get('https://httpbin.org/user-agent')
useragent = json.loads(r.text)['user-agent']
headers = {'User-agent': useragent}

In [77]:
import googlemaps
from datetime import datetime, timedelta
import dotenv
import json
import pandas as pd

dotenv.load_dotenv()
api_key_google = os.getenv('api_key') 

gmaps = googlemaps.Client(key=api_key_google)

# Define the origin and destination
origin = "Richmond, VA"
destination = "Pittsburgh, PA"

# Define parameters for the directions request
params = {
    'origin': origin,
    'destination': destination,
    'mode': 'driving',  # You can change this to 'walking', 'bicycling', or 'transit'
    'alternatives': True,  # If you want alternative routes
    'avoid': None,  # You can specify features to avoid, e.g., 'tolls', 'highways'
    'language': 'en',  # Language for the results
    'units': 'imperial',  # Use 'metric' for metric units
    'departure_time': datetime.now(),  # Use the current time as the departure time
}

# Fetch directions
directions = gmaps.directions(**params)

# Check if directions were returned
if directions:
    # Print the total number of routes found
    print(f'Total routes found: {len(directions)}')
    
    for idx, route in enumerate(directions):
        print(f"\nRoute {idx + 1}:")
        print(f"  Distance: {route['legs'][0]['distance']['text']}")
        print(f"  Duration: {route['legs'][0]['duration']['text']}")
        print(f"  Start Address: {route['legs'][0]['start_address']}")
        print(f"  End Address: {route['legs'][0]['end_address']}")
        
        print("\n  Directions:")
        for step in route['legs'][0]['steps']:
            print(f"    - {step['html_instructions']} (Approx. {step['duration']['text']})")
else:
    print("No directions found.")

Total routes found: 3

Route 1:
  Distance: 345 mi
  Duration: 5 hours 22 mins
  Start Address: Richmond, VA, USA
  End Address: Pittsburgh, PA, USA

  Directions:
    - Head <b>northwest</b> on <b>E Grace St</b> toward <b>N 7th St</b> (Approx. 1 min)
    - Turn <b>right</b> at the 1st cross street onto <b>N 7th St</b> (Approx. 2 mins)
    - Slight <b>right</b> onto the <b>I-64 W</b>/<wbr/><b>I-95 N</b> ramp (Approx. 1 min)
    - Merge onto <b>I-95 N</b> (Approx. 4 mins)
    - Keep <b>left</b> to stay on <b>I-95 N</b> (Approx. 49 mins)
    - Slight <b>left</b> (Approx. 1 min)
    - Continue onto <b>I-95 Express Lanes</b><div style="font-size:0.9em">Toll road</div><div style="font-size:0.9em">May be closed at certain times or days</div> (Approx. 7 mins)
    - Continue onto <b>I-95 Express Lanes</b><div style="font-size:0.9em">Toll road</div><div style="font-size:0.9em">May be closed at certain times or days</div> (Approx. 22 mins)
    - Take the <b>I-95 N</b>/<wbr/><b>I-495</b> exit tow

In [83]:
news_df = pd.json_normalize(directions[0], record_path=['legs', 'steps'])

news_df['time_in_minutes'] = round(news_df['duration.value']/60, 0)
news_df['time_in_minutes'] = news_df['time_in_minutes'].astype('int')
news_df['cumulative_time'] = news_df['time_in_minutes'].cumsum()

news_df['date_time'] = news_df['cumulative_time'].apply(lambda x: datetime.now() + timedelta(minutes=x))
news_df['date_time'] = news_df['date_time'].apply(lambda x: x.replace(minute=0, second=0, microsecond=0))

max_time = news_df['cumulative_time'].iloc[(len(news_df)-1)]

desired_val = list(range(0,max_time+60, 60))
news_df1 = news_df[['end_location.lat', 'end_location.lng', 'cumulative_time', 'date_time']]
closest_rows = (news_df1.loc[[abs(news_df1['cumulative_time'] - hour).idxmin() for hour in desired_val]]).reset_index()

In [86]:
news_df

,html_instructions,travel_mode,distance.text,distance.value,duration.text,duration.value,end_location.lat,end_location.lng,polyline.points,start_location.lat,start_location.lng,maneuver,time_in_minutes,cumulative_time,date_time
0,Head <b>northwest</b> on <b>E Grace St</b> tow...,DRIVING,161 ft,49,1 min,11,37.541017,-77.436471,udcdFbfswMCDq@rA,37.540754,-77.436020,NaN,0,0,2024-10-20 19:00:00
1,Turn <b>right</b> at the 1st cross street onto...,DRIVING,0.6 mi,939,2 mins,133,37.547736,-77.430052,kfcdF|hswMwAuA[Yi@g@UUwAoASQu@s@q@m@W]e@e@yAwA...,37.541017,-77.436471,turn-right,2,2,2024-10-20 19:00:00
2,Slight <b>right</b> onto the <b>I-64 W</b>/<wb...,DRIVING,0.2 mi,347,1 min,32,37.547750,-77.431249,kpddFx`rwMa@kAEe@?K?E@I@CBKDK@CBEDEJEDC@?HA@?L...,37.547736,-77.430052,ramp-right,1,3,2024-10-20 19:00:00
3,Merge onto <b>I-95 N</b>,DRIVING,3.4 mi,5482,4 mins,218,37.581473,-77.469396,mpddFhhrwM_@t@e@~@Yh@q@rAMT]r@Ub@KRMVMTINADMTK...,37.547750,-77.431249,merge,4,7,2024-10-20 19:00:00
4,Keep <b>left</b> to stay on <b>I-95 N</b>,DRIVING,55.4 mi,89121,49 mins,2911,38.347349,-77.483722,eckdFvvywM_@LQDUH}@To@LOBWDQBI@M@A?K@I@UBS@_@B...,37.581473,-77.469396,keep-left,49,56,2024-10-20 20:00:00
5,Slight <b>left</b>,DRIVING,0.1 mi,234,1 min,7,38.349003,-77.482086,}u`iFfp|wMOCA?SWg@i@EGA?UWCCqBqB[[?AkAkA,38.347349,-77.483722,turn-slight-left,0,56,2024-10-20 20:00:00
6,Continue onto <b>I-95 Express Lanes</b><div st...,DRIVING,9.1 mi,14713,7 mins,435,38.463491,-77.408396,g`aiF`f|wMOO{AgBkByBs@y@q@u@u@_ACCUYCEUY]c@EGS...,38.349003,-77.482086,NaN,7,63,2024-10-20 20:00:00
7,Continue onto <b>I-95 Express Lanes</b><div st...,DRIVING,26.9 mi,43214,22 mins,1337,38.788942,-77.176814,ykwiFnymwMiBg@OEq@UEAMEOEAA]Ma@OUIsAk@YMs@YUM]...,38.463491,-77.408396,NaN,22,85,2024-10-20 21:00:00
8,Take the <b>I-95 N</b>/<wbr/><b>I-495</b> exit...,DRIVING,0.5 mi,796,1 min,36,38.792299,-77.170960,{}vkF`r`vMIM?AQG]Kc@S[OYO[QKG_@WGECCQMWSGEWUYY...,38.788942,-77.176814,ramp-right,1,86,2024-10-20 21:00:00
9,"At the interchange <b>I-495 N</b>, follow sign...",DRIVING,0.9 mi,1497,1 min,63,38.795069,-77.185837,{rwkFnm_vMTA\CB?L?@?@?B?JBRB@?JDLB@@@?FDJDJHJH...,38.792299,-77.170960,ramp,1,87,2024-10-20 21:00:00


In [88]:
news_df.iloc[19]['polyline.points']

'ajasFles|MJV@B@DHRBHBFHTHPDJBHFPZv@@DHPb@jABDNb@FPBBXr@BJFLJXBFFLBHFNFNDLDLFPBHFPDRBHDJBN@@Jj@DN?@DVBRDZBPD\\Jt@JhAPxABVLVD\\BXB^DXBZDZDZD^@J@JFZDZDZDXDZDXFZDZF\\DXDTF^Lv@?BDRFZFZDXFXFZFZDXFXFZLp@FZFZFXFZF`@DNF\\FZDXFXFXDXBJPbAH\\DXFZFVDZFZFXDXFZFZ?@FXDP@FDXFXFZFZDXFZDVFXFZBLJj@F\\Lj@Nv@?@F\\FXFZTlANv@FTDT@DF\\FXDTFZDR?BPv@F\\F\\DVDXFZDZFZ@LBLBXF\\NtAHfA@FDf@Dr@Dr@Dv@Bx@@t@Bx@@v@?v@?X?f@AhA?V?D?d@?lAAv@?\\?\\?H?R?nAAz@?x@?@?t@?|@ApA?`@?l@?^?\\Ap@?^?j@?L?^A\\?VAX?`@Ct@Cl@ATCd@Ef@IbAGf@Gl@Ih@Il@Kr@GZEXI\\S~@EREPIV?BMb@Md@KZITK\\Sr@Up@KZ?BQf@IVIVK^_@hASr@IVKXSp@Ql@K^Qp@Qr@GXMl@I`@Mv@Kp@EVEZAHMfA?DCT?@CZCXADCZAVCZAZCj@EfA?XAb@Cp@AZIvC?VAVCv@AZCtAAZAZA\\AZAZCx@?ZA^AXCv@Al@Cn@Cn@Cl@AHA\\CXC\\EZCZEXCTAFEZAFEVETEXAHEPABETET?@IZIXI\\GTIXKXIVIVKXM\\GRCDGNIXKVKVKXIVKVIXWn@IXWn@Up@IVO^GPKVADGPUp@KVKVOd@Qb@ABGRIRABIXa@fAIRADITEJQd@IVGPGNKVEPKT?@KVIVIXCHA@ELAFGRGTIXAHENIZEXCFCNEZG^ETEZCZEV?FCTCZCZCl@AF?@AZAX?^AZ?X?b@?VAd@?jA?v@AjD?Z?Z?z@?t@A`A?V?\\?Z?Z?\\?Z?F?R?ZAZ?\\?ZAZ?NAJAZA\\CZAZCZEZCZEXGZEZGX

In [79]:
closest_rows['date_time'] = pd.to_datetime(closest_rows['date_time'])
closest_rows = closest_rows.drop(['index'], axis=1)
closest_rows


,end_location.lat,end_location.lng,cumulative_time,date_time
0,37.541017,-77.436471,0,2024-10-20 19:00:00
1,38.463491,-77.408396,63,2024-10-20 20:00:00
2,39.037024,-77.144951,107,2024-10-20 21:00:00
3,39.998866,-78.238898,204,2024-10-20 23:00:00
4,39.988968,-78.255108,207,2024-10-20 23:00:00
5,40.434320,-79.748054,303,2024-10-21 00:00:00
6,40.440635,-79.995913,323,2024-10-21 01:00:00


In [80]:
temp_list = []
precip_chance_list = []
wind_speed_list = []
skycover_list = []
snowfall_list = []
windgust_list = []


for i in range(len(closest_rows)):

    desired_val = [closest_rows['date_time'][i]]

    url = f"https://api.weather.gov/points/{closest_rows['end_location.lat'][i]},{closest_rows['end_location.lng'][i]}"
    r = requests.get(url, headers = headers)

    myjson = json.loads(r.text)
    df_url_info = pd.json_normalize(myjson['properties'])

    hourlyURL = df_url_info['forecastHourly'].iloc[0]   
    hourlyURL_grid = df_url_info['forecastGridData'].iloc[0] 

    # Obtain actual hourly forecast data

    r = requests.get(hourlyURL, headers = headers)
    r_g = requests.get(hourlyURL_grid, headers = headers)

    myjson = json.loads(r.text)
    myjson_g = json.loads(r_g.text)

    df = pd.json_normalize(myjson['properties']['periods'])
    df['startTime'] = pd.to_datetime(df['startTime']).dt.tz_localize(None)

    df = (df.loc[[abs(df['startTime'] - hour).idxmin() for hour in desired_val]]).reset_index()

    df_temp = df['temperature']
    df_pop = df['probabilityOfPrecipitation.value']
    df_ws = df['windSpeed']

    temp_list.append(df_temp)
    precip_chance_list.append(df_pop)
    wind_speed_list.append(df_ws)

    # Obtain actual hourly forecast data from grid forecast

    df_sky = pd.json_normalize(myjson_g['properties']['skyCover']['values'])
    df_wg = pd.json_normalize(myjson_g['properties']['windGust']['values'])
    df_snow = pd.json_normalize(myjson_g['properties']['snowfallAmount']['values'])
    

    def convert_time_select_closest_row(var_name):

        var_name['validTime'] = var_name['validTime'].str.extract(r'^(.*?)/')
        var_name['validTime'] = pd.to_datetime(var_name['validTime'])
        var_name['validTime'] = var_name['validTime'].dt.tz_localize(None)


        df_ab = (var_name.loc[[abs(var_name['validTime'] - hour).idxmin() for hour in desired_val]]).reset_index()

        return df_ab
    
    df_skycover = convert_time_select_closest_row(df_sky)
    df_skycover = df_skycover['value']

    df_snowfall = convert_time_select_closest_row(df_snow)
    df_snowfall = df_snowfall['value']

    df_windgust = convert_time_select_closest_row(df_wg)
    df_windgust = df_windgust['value']


    skycover_list.append(df_skycover)
    snowfall_list.append(df_snowfall)
    windgust_list.append(df_windgust)


temp_list = np.array(temp_list)

precip_chance_list = np.array(precip_chance_list)

wind_speed_list = np.array(wind_speed_list)
wind_speed_list = wind_speed_list.flatten()

skycover_list = np.array(skycover_list)
skycover_list = skycover_list.flatten()

snowfall_list = np.array(snowfall_list)
snowfall_list = snowfall_list.flatten()

windgust_list = np.array(windgust_list)

closest_rows['temp'] = temp_list
closest_rows['precip'] = precip_chance_list
closest_rows['ws'] = wind_speed_list
closest_rows['skycover'] = skycover_list
closest_rows['snowfall'] = snowfall_list
closest_rows['wg'] = windgust_list


In [81]:
closest_rows

,end_location.lat,end_location.lng,cumulative_time,date_time,temp,precip,ws,skycover,snowfall,wg
0,37.541017,-77.436471,0,2024-10-20 19:00:00,64,0,2 mph,1,0,9.260
1,38.463491,-77.408396,63,2024-10-20 20:00:00,56,0,1 mph,2,0,5.556
2,39.037024,-77.144951,107,2024-10-20 21:00:00,57,0,2 mph,1,0,9.260
3,39.998866,-78.238898,204,2024-10-20 23:00:00,47,0,1 mph,2,0,7.408
4,39.988968,-78.255108,207,2024-10-20 23:00:00,47,0,1 mph,2,0,7.408
5,40.434320,-79.748054,303,2024-10-21 00:00:00,49,0,3 mph,0,0,12.964
6,40.440635,-79.995913,323,2024-10-21 01:00:00,50,0,2 mph,0,0,9.260


In [18]:
url = f"https://api.weather.gov/points/{35},{-90}"
r = requests.get(url, headers = headers)

In [19]:
myjson = json.loads(r.text)
df_url_info = pd.json_normalize(myjson['properties'])

In [20]:
df_url_info

,@id,@type,cwa,forecastOffice,gridId,gridX,gridY,forecast,forecastHourly,forecastGridData,...,radarStation,relativeLocation.type,relativeLocation.geometry.type,relativeLocation.geometry.coordinates,relativeLocation.properties.city,relativeLocation.properties.state,relativeLocation.properties.distance.unitCode,relativeLocation.properties.distance.value,relativeLocation.properties.bearing.unitCode,relativeLocation.properties.bearing.value
0,"https://api.weather.gov/points/35,-90",wx:Point,MEG,https://api.weather.gov/offices/MEG,MEG,44,60,"https://api.weather.gov/gridpoints/MEG/44,60/f...","https://api.weather.gov/gridpoints/MEG/44,60/f...","https://api.weather.gov/gridpoints/MEG/44,60",...,KNQA,Feature,Point,"[-89.977669, 34.951005]",Southaven,MS,wmoUnit:m,5815.540383,wmoUnit:degree_(angle),339


In [21]:
hourlyURL = df_url_info['forecastHourly'].iloc[0]   
hourlyURL_grid = df_url_info['forecastGridData'].iloc[0]

In [22]:
hourlyURL

'https://api.weather.gov/gridpoints/MEG/44,60/forecast/hourly'

In [26]:
hourlyURL_grid

'https://api.weather.gov/gridpoints/MEG/44,60'

In [24]:
r = requests.get(hourlyURL, headers = headers)
r_g = requests.get(hourlyURL_grid, headers = headers)

myjson = json.loads(r.text)
myjson_g = json.loads(r_g.text)

df = pd.json_normalize(myjson['properties']['periods'])

In [43]:
df1 = pd.json_normalize(myjson_g['properties'])
df1

,@id,@type,updateTime,validTimes,forecastOffice,gridId,gridX,gridY,elevation.unitCode,elevation.value,...,potentialOf40mphWindGusts.values,potentialOf50mphWindGusts.values,potentialOf60mphWindGusts.values,grasslandFireDangerIndex.values,probabilityOfThunder.values,davisStabilityIndex.values,atmosphericDispersionIndex.values,lowVisibilityOccurrenceRiskIndex.values,stability.values,redFlagThreatIndex.values
0,"https://api.weather.gov/gridpoints/LWX/83,69",wx:Gridpoint,2024-10-14T23:30:20+00:00,2024-10-14T17:00:00+00:00/P7DT8H,https://api.weather.gov/offices/LWX,LWX,83,69,wmoUnit:m,91.1352,...,[],[],[],[],[{'validTime': '2024-10-14T17:00:00+00:00/P1DT...,[],[{'validTime': '2024-10-14T17:00:00+00:00/PT1H...,[{'validTime': '2024-10-14T17:00:00+00:00/PT6H...,[],[]


In [53]:
df_sky = pd.json_normalize(myjson_g['properties']['skyCover']['values'])


In [56]:
df_sky

,validTime,value
0,2024-10-14T17:00:00+00:00/PT1H,51
1,2024-10-14T18:00:00+00:00/PT1H,47
2,2024-10-14T19:00:00+00:00/PT1H,46
3,2024-10-14T20:00:00+00:00/PT1H,44
4,2024-10-14T21:00:00+00:00/PT1H,43
...,...,...
112,2024-10-21T12:00:00+00:00/PT5H,5
113,2024-10-21T17:00:00+00:00/PT3H,4
114,2024-10-21T20:00:00+00:00/PT2H,5
115,2024-10-21T22:00:00+00:00/PT1H,6


In [67]:
df_sky['validTime'] = df_sky['validTime'].str.extract(r'^(.*?)/')
df_sky




,validTime,value
0,NaN,19
1,NaN,22
2,NaN,31
3,NaN,23
4,NaN,15
...,...,...
100,NaN,10
101,NaN,11
102,NaN,12
103,NaN,13


In [70]:
df_sky

,validTime,value
0,2024-10-14T16:00:00+00:00,19
1,2024-10-14T17:00:00+00:00,22
2,2024-10-14T18:00:00+00:00,31
3,2024-10-14T19:00:00+00:00,23
4,2024-10-14T20:00:00+00:00,15
...,...,...
100,2024-10-22T08:00:00+00:00,10
101,2024-10-22T09:00:00+00:00,11
102,2024-10-22T10:00:00+00:00,12
103,2024-10-22T11:00:00+00:00,13


In [4]:
start_lat = 43.6568
start_lng = -79.4512
end_lat = 39.9526
end_lng = -75.1652

In [5]:
import requests
import datetime

# Define the Mapbox API endpoint and your access token
MAPBOX_ACCESS_TOKEN = 'pk.eyJ1IjoiZmlyc3RpbndlYXRoZXIiLCJhIjoiY20ydjlpY215MDl4NjJqb2l1ZjBwbXo2NSJ9.vt3Xx08GULpig9DYBb5o0A'
start_point = [start_lng, start_lat]  # Example: Start coordinate (lon, lat)
end_point = [end_lng, end_lat]    # Example: End coordinate (lon, lat)

# Request the route using the Mapbox Directions API
url = f"https://api.mapbox.com/directions/v5/mapbox/driving/{start_point[0]},{start_point[1]};{end_point[0]},{end_point[1]}"
params = {
    'geometries': 'geojson',
    'overview': 'full',
    'steps': 'true',
    'access_token': MAPBOX_ACCESS_TOKEN
}
response = requests.get(url, params=params)
data = response.json()

# Parse the response to get duration and coordinates
route = data['routes'][0]
total_duration = route['duration']  # Total duration in seconds
interval = 1800  # 30 minutes in seconds

# Extract waypoints at 30-minute intervals
coordinates_every_30_minutes = []
current_time = 0

for leg in route['legs']:
    for step in leg['steps']:
        
        step_duration = step['duration']
        step_coords = step['geometry']['coordinates']
        
        for i, coord in enumerate(step_coords):
            # Calculate cumulative travel time to this point
            cumulative_time = current_time + (step_duration / len(step_coords)) * i
            
            # If the cumulative time is close to the next interval, add the coordinate
            if cumulative_time >= interval * len(coordinates_every_30_minutes):
                coordinates_every_30_minutes.append({
                    'lat': coord[1],
                    'lon': coord[0],
                    'time': str(datetime.timedelta(seconds=cumulative_time))
                })
        
        current_time += step_duration  # Update time for the next step

# Display the coordinates with timestamps every 30 minutes
for point in coordinates_every_30_minutes:
    print(f"Lat: {point['lat']}, Lon: {point['lon']}, Time: {point['time']}")


Lat: 43.656583, Lon: -79.451111, Time: 0:00:00
Lat: 43.497766, Lon: -79.673875, Time: 0:30:03.338106
Lat: 43.22386, Lon: -79.665253, Time: 1:00:04.982437
Lat: 43.146982, Lon: -79.103275, Time: 1:30:00.338387
Lat: 42.997189, Lon: -78.807809, Time: 2:00:01.041259
Lat: 43.018935, Lon: -78.181262, Time: 2:30:01.244459
Lat: 43.014307, Lon: -77.471497, Time: 3:00:00.680800
Lat: 42.996027, Lon: -76.737292, Time: 3:30:00.117141
Lat: 43.105944, Lon: -76.284268, Time: 4:00:00.962200
Lat: 42.802158, Lon: -76.122229, Time: 4:30:03.515167
Lat: 42.32069, Lon: -75.970943, Time: 5:00:02.922383
Lat: 41.958916, Lon: -75.737123, Time: 5:30:01.298730
Lat: 41.610587, Lon: -75.6453, Time: 6:00:02.803000
Lat: 41.31543, Lon: -75.744052, Time: 6:30:01.012095
Lat: 40.878154, Lon: -75.65743, Time: 7:00:01.608467
Lat: 40.494877, Lon: -75.479687, Time: 7:30:02.204840
Lat: 40.069397, Lon: -75.27988, Time: 8:00:00.654642


In [77]:
route
route_df = pd.json_normalize(route, record_path=['legs', 'steps'])

In [80]:
a = route['legs']

In [81]:
b = pd.json_normalize(a)

In [82]:
b

,notifications,via_waypoints,admins,weight,duration,steps,distance,summary
0,"[{'details': {'actual_value': 'CA,US', 'messag...",[],"[{'iso_3166_1_alpha3': 'CAN', 'iso_3166_1': 'C...",31092.727,30484.287,"[{'intersections': [{'entry': [True], 'bearing...",796777.938,"I 90 East, I 81 South"


In [86]:
for leg in route['legs']:
    print(1)

1


In [89]:
coordinates_every_30_minutes

[{'lat': 43.656583, 'lon': -79.451111, 'time': '0:00:00'},
 {'lat': 43.497766, 'lon': -79.673875, 'time': '0:30:03.338106'},
 {'lat': 43.22386, 'lon': -79.665253, 'time': '1:00:04.982437'},
 {'lat': 43.146982, 'lon': -79.103275, 'time': '1:30:00.338387'},
 {'lat': 42.997189, 'lon': -78.807809, 'time': '2:00:01.041259'},
 {'lat': 43.018935, 'lon': -78.181262, 'time': '2:30:01.197123'},
 {'lat': 43.014307, 'lon': -77.471497, 'time': '3:00:00.486800'},
 {'lat': 42.996473, 'lon': -76.735788, 'time': '3:30:04.536502'},
 {'lat': 43.105944, 'lon': -76.284268, 'time': '4:00:00.477200'},
 {'lat': 42.802158, 'lon': -76.122229, 'time': '4:30:03.064744'},
 {'lat': 42.32069, 'lon': -75.970943, 'time': '5:00:02.522472'},
 {'lat': 41.958916, 'lon': -75.737123, 'time': '5:30:00.926730'},
 {'lat': 41.610587, 'lon': -75.6453, 'time': '6:00:02.431000'},
 {'lat': 41.31543, 'lon': -75.744052, 'time': '6:30:00.640095'},
 {'lat': 40.878154, 'lon': -75.65743, 'time': '7:00:01.236467'},
 {'lat': 40.494877, 'lo

In [90]:
c = pd.DataFrame(coordinates_every_30_minutes)
c

,lat,lon,time
0,43.656583,-79.451111,0:00:00
1,43.497766,-79.673875,0:30:03.338106
2,43.223860,-79.665253,1:00:04.982437
3,43.146982,-79.103275,1:30:00.338387
4,42.997189,-78.807809,2:00:01.041259
5,43.018935,-78.181262,2:30:01.197123
6,43.014307,-77.471497,3:00:00.486800
7,42.996473,-76.735788,3:30:04.536502
8,43.105944,-76.284268,4:00:00.477200
9,42.802158,-76.122229,4:30:03.064744


In [ ]:
https://api.mapbox.com/directions/v5/mapbox/driving/43,-73,45,-76

In [13]:
#import datetime
from datetime import datetime, timedelta

In [17]:
selected_departure_time = datetime.now()
simplified_today = selected_departure_time.replace(minute=0, second=0, microsecond=0)

simplified_today = (simplified_today, "%Y-%m-%d %H:%M")
a = selected_departure_time + timedelta(hours=3)
a

datetime.datetime(2024, 11, 2, 1, 18, 47, 19070)

In [18]:
print(a)

2024-11-02 01:18:47.019070


In [2]:
### Open Weather API

In [21]:
r = requests.get('https://httpbin.org/user-agent')
useragent = json.loads(r.text)['user-agent']
headers = {'User-agent': useragent}

API_key = '6e4a8336ddea630116c32b827c5226be'

from datetime import datetime, timezone

date = "2024-11-09 12:00:00"
date_object = datetime.strptime(date, "%Y-%m-%d %H:%M:%S").replace(tzinfo=timezone.utc)
unix_timestamp = int(date_object.timestamp())
print("Unix timestamp:", unix_timestamp)

Unix timestamp: 1731153600


In [22]:
url = f'https://api.openweathermap.org/data/3.0/onecall/timemachine?lat={34}&lon={-77}&dt={unix_timestamp}&appid={API_key}'

In [23]:
url

'https://api.openweathermap.org/data/3.0/onecall/timemachine?lat=34&lon=-77&dt=1731153600&appid=6e4a8336ddea630116c32b827c5226be'

In [24]:
r = requests.get(url, headers = headers)
myjson = json.loads(r.text)

In [25]:
myjson

{'lat': 34,
 'lon': -77,
 'timezone': 'Etc/GMT+5',
 'timezone_offset': -18000,
 'data': [{'dt': 1731153600,
   'sunrise': 1731152122,
   'sunset': 1731190108,
   'temp': 294.75,
   'feels_like': 294.92,
   'pressure': 1020,
   'humidity': 75,
   'dew_point': 290.13,
   'uvi': 0,
   'clouds': 33,
   'visibility': 10000,
   'wind_speed': 11.94,
   'wind_deg': 40,
   'wind_gust': 11.92,
   'weather': [{'id': 802,
     'main': 'Clouds',
     'description': 'scattered clouds',
     'icon': '03d'}]}]}

In [31]:
df = pd.json_normalize(myjson['data'])
df

,dt,sunrise,sunset,temp,feels_like,pressure,humidity,dew_point,uvi,clouds,visibility,wind_speed,wind_deg,wind_gust,weather
0,1731153600,1731152122,1731190108,294.75,294.92,1020,75,290.13,0,33,10000,11.94,40,11.92,"[{'id': 802, 'main': 'Clouds', 'description': ..."


In [32]:
df['weather']

0    [{'id': 802, 'main': 'Clouds', 'description': ...
Name: weather, dtype: object

In [61]:
url = f'https://api.openweathermap.org/data/3.0/onecall?lat=42.44&lon=-71.04&exclude=current,minutely,daily&appid={API_key}'

In [62]:
r = requests.get(url, headers = headers)
myjson = json.loads(r.text)

In [64]:
df = pd.json_normalize(myjson['hourly'])
df

,dt,temp,feels_like,pressure,humidity,dew_point,uvi,clouds,visibility,wind_speed,wind_deg,wind_gust,weather,pop,rain.1h
0,1731211200,275.83,274.17,1029,54,268.10,0.00,0,10000,1.71,326,2.95,"[{'id': 800, 'main': 'Clear', 'description': '...",0.0,NaN
1,1731214800,276.30,276.30,1029,50,267.60,0.00,0,10000,1.21,317,1.90,"[{'id': 800, 'main': 'Clear', 'description': '...",0.0,NaN
2,1731218400,276.69,276.69,1029,48,267.44,0.00,0,10000,0.94,283,1.32,"[{'id': 800, 'main': 'Clear', 'description': '...",0.0,NaN
3,1731222000,276.99,276.99,1029,45,266.94,0.00,1,10000,1.24,264,1.88,"[{'id': 800, 'main': 'Clear', 'description': '...",0.0,NaN
4,1731225600,277.10,276.03,1028,43,266.50,0.00,2,10000,1.41,245,2.57,"[{'id': 800, 'main': 'Clear', 'description': '...",0.0,NaN
5,1731229200,277.23,275.50,1028,42,265.42,0.00,2,10000,1.95,245,3.67,"[{'id': 800, 'main': 'Clear', 'description': '...",0.0,NaN
6,1731232800,277.02,275.11,1027,42,265.26,0.00,2,10000,2.09,236,4.89,"[{'id': 800, 'main': 'Clear', 'description': '...",0.0,NaN
7,1731236400,276.80,274.86,1027,41,264.87,0.00,4,10000,2.08,240,5.41,"[{'id': 800, 'main': 'Clear', 'description': '...",0.0,NaN
8,1731240000,276.94,274.64,1027,39,264.27,0.00,7,10000,2.47,232,6.66,"[{'id': 800, 'main': 'Clear', 'description': '...",0.0,NaN
9,1731243600,278.25,275.77,1027,35,264.09,0.31,45,10000,3.01,230,7.82,"[{'id': 802, 'main': 'Clouds', 'description': ...",0.0,NaN


In [65]:
df['dt'] = df['dt'].apply(lambda x: datetime.fromtimestamp(x, tz=timezone.utc))


In [66]:
df

,dt,temp,feels_like,pressure,humidity,dew_point,uvi,clouds,visibility,wind_speed,wind_deg,wind_gust,weather,pop,rain.1h
0,2024-11-10 04:00:00+00:00,275.83,274.17,1029,54,268.10,0.00,0,10000,1.71,326,2.95,"[{'id': 800, 'main': 'Clear', 'description': '...",0.0,NaN
1,2024-11-10 05:00:00+00:00,276.30,276.30,1029,50,267.60,0.00,0,10000,1.21,317,1.90,"[{'id': 800, 'main': 'Clear', 'description': '...",0.0,NaN
2,2024-11-10 06:00:00+00:00,276.69,276.69,1029,48,267.44,0.00,0,10000,0.94,283,1.32,"[{'id': 800, 'main': 'Clear', 'description': '...",0.0,NaN
3,2024-11-10 07:00:00+00:00,276.99,276.99,1029,45,266.94,0.00,1,10000,1.24,264,1.88,"[{'id': 800, 'main': 'Clear', 'description': '...",0.0,NaN
4,2024-11-10 08:00:00+00:00,277.10,276.03,1028,43,266.50,0.00,2,10000,1.41,245,2.57,"[{'id': 800, 'main': 'Clear', 'description': '...",0.0,NaN
5,2024-11-10 09:00:00+00:00,277.23,275.50,1028,42,265.42,0.00,2,10000,1.95,245,3.67,"[{'id': 800, 'main': 'Clear', 'description': '...",0.0,NaN
6,2024-11-10 10:00:00+00:00,277.02,275.11,1027,42,265.26,0.00,2,10000,2.09,236,4.89,"[{'id': 800, 'main': 'Clear', 'description': '...",0.0,NaN
7,2024-11-10 11:00:00+00:00,276.80,274.86,1027,41,264.87,0.00,4,10000,2.08,240,5.41,"[{'id': 800, 'main': 'Clear', 'description': '...",0.0,NaN
8,2024-11-10 12:00:00+00:00,276.94,274.64,1027,39,264.27,0.00,7,10000,2.47,232,6.66,"[{'id': 800, 'main': 'Clear', 'description': '...",0.0,NaN
9,2024-11-10 13:00:00+00:00,278.25,275.77,1027,35,264.09,0.31,45,10000,3.01,230,7.82,"[{'id': 802, 'main': 'Clouds', 'description': ...",0.0,NaN
